In [8]:
pip install thorpy

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [9]:
import math
import pygame
import thorpy as tp

# Constants
SCREEN_SIZE = (1200, 600)
LINE_CENTER_COORD = (100,500)
LINE_CENTER_X = LINE_CENTER_COORD[0]
LINE_CENTER_Y = LINE_CENTER_COORD[1]
LINE_X_L = 500
SQUARE_SIDE_L = 50

pygame.init()

surface = pygame.Surface((10, 10))
screen = pygame.display.set_mode(SCREEN_SIZE)
clock = pygame.time.Clock()
tp.init(screen, tp.theme_human)

my_button = tp.Button("Hello, world.\nThis button uses the default theme.")
my_button.center_on(screen)
theta = tp.SliderWithText("Theta(θ)", 15, 45, 30, 200)
theta.set_bottomright(400,50)
mass = tp.SliderWithText("Mass", 30, 60, 45, 200)
mass.set_topleft(400,100)
slider4 = tp.SliderWithText("How many games should we play ?",
                                1, 10, 3, #min, max and initial values
                                50, "h", #length and orientation
                                dragger_size=(10,20),
                                show_value_on_right_side=True,
                                edit=True) #allow to edit value as a text
slider5 = tp.SliderWithText("Fine tune some number\n(from -2 to 2)", -2, 2, 1.5, 200,
                                round_decimals=3) #by default, 2 decimals
surfaceMaterialsDD = tp.DropDownListButton(
    choices= ["no friction", "ice", "rocky"],
    title="Surface Type",
    generate_shadow= (False, 'auto')
)

group = tp.Group([theta, mass, slider5, my_button, surfaceMaterialsDD])
group.set_topleft(700,50)

rect_surface = pygame.Surface((SQUARE_SIDE_L,)*2, pygame.SRCALPHA)
rect_surface.fill((120,)*3)

def before_gui(): # Main loop, this approach is used because of thorPy library
    screen.fill((250,)*3)
    
    hypotenuse = LINE_X_L*math.cos(theta.get_value()*(math.pi/180))
    ending_point_Y = LINE_CENTER_Y - (hypotenuse*math.sin(theta.get_value()*(math.pi/180)))
    ending_point_X = LINE_CENTER_X + (hypotenuse*math.cos(theta.get_value()*(math.pi/180)))

    
    rect_surface_rot = pygame.transform.rotate(rect_surface, theta.get_value())
    rect_center = rect_surface_rot.get_rect(center=(ending_point_X - SQUARE_SIDE_L // 2, ending_point_Y - SQUARE_SIDE_L // 2 +(5*(theta.get_value()/15))))  # +(5*(theta.get_value()/15))
    
    screen.blit(rect_surface_rot, rect_center)

    pygame.draw.line(screen, (10,130,10), (0, LINE_CENTER_Y), (LINE_CENTER_X+ LINE_X_L, LINE_CENTER_Y), 10)
    pygame.draw.line(screen, (0,0,0), LINE_CENTER_COORD, (ending_point_X, ending_point_Y), 10)
    pygame.draw.circle(screen, (165,42,42), LINE_CENTER_COORD, 10)

tp.call_before_gui(before_gui) # tells thorpy to call before_gui() before drawing gui.


player = group.get_updater().launch(before_gui)

pygame.quit()